In [ ]:
#Import Packages
#Import/Install Packages
import datetime
import os
import requests
import time
      
!pip install --quiet geopandas
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
plt.style.use('classic')
%matplotlib inline
import numpy as np
requests.packages.urllib3.disable_warnings(requests.packages.urllib3.exceptions.InsecureRequestWarning)

In [2]:
#Prep for API Call
# Set root URL for API requests
root_url = 'https://api.climateengine.org/'
# Authentication info for the API (INSERT YOUR OWN KEY)
headers = {'Authorization': 'INSERT YOUR API KEY HERE'}

In [ ]:
#Bring in Snotel lat/longs: https://wcc.sc.egov.usda.gov/nwcc/yearcount?network=sntl&state=&counttype=statelist
data = pd.read_csv('/content/SnotelStationLocations.csv')
data = data[data['state']!= 'AK']
siteName = pd.unique(data[['site_name']].values.ravel('k'))
print('Site Name:', siteName)
latValue = pd.unique(data[['lat']].values.ravel('k')).astype(float)
print('Latitude Value:',latValue)
longValue = pd.unique(data[['lon']].values.ravel('k')).astype(float)
print('Longitude Value:',longValue)

In [ ]:
#Create a coordinate list in a format the API request can use
coordinateList = [list(x) for x in zip(longValue, latValue)]
#coordinateList = str(coordinateList)
sites = {}
for x in siteName:
  for y in coordinateList:
    sites[x] = str([y])
    coordinateList.remove(y)
    break
print(sites)

In [ ]:
# Mount Google Drive for exporting CSVs
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Write function to automate export of csvs and pngs of SNODAS data to Google Drive
def write_timeseries(siteName, dataset, variable, start_date, end_date, variableC):
  
  print("Attempting export for " + siteName)

  # Endpoint
  endpoint1 = 'timeseries/native/points'

  # Set up parameters for API call
  params1 = {
      'dataset': dataset,
      'variable': variable,
      'start_date': start_date ,
      'end_date': end_date,
      'coordinates': sites[siteName],
      'area_reducer':'mean'
  }

  # Send request to the API
  r = requests.get(root_url + endpoint1, params=params1, headers=headers, verify=True)
  response = r.json()

  #response (may need to unpack with [] around timeseries the first time)
  [timeseries] = response

  #Select data
  data = timeseries['Data']

  # Convert to dataframe
  df = pd.DataFrame.from_dict(data)

  #Process timeseries data 
  #Filter out non-available data i.e. values of -9999.000
  df = df[df[variableC]>-100]

  #Format dates for plotting
  df['Date'] = pd.to_datetime(df['Date'])

  #Set x values to date variable
  date = df['Date']

  #Set y values to value variable
  value = df[variableC]

  #Plot timeseries data
  #Generate graph of long term blend values across date range
  fig, ax = plt.subplots(figsize=(18,6))
  name = siteName
  ax.plot(date, value, label = name)  

  #Add x-axis label
  plt.xlabel("Dates")
                      
  #Add y-axis label
  plt.ylabel(variableC)
           
  #Add Title
  plt.title(dataset + " " + variable  + " " + start_date +" through " + end_date)

  #Add Legend 
  plt.legend(loc='upper left')


  # # Conditionally create Google Drive directories to store outputs, if they don't exist 
  if not os.path.exists("/content/drive/MyDrive/Snotel_Example/"):
    os.mkdir("/content/drive/MyDrive/Snotel_Example/")
  if not os.path.exists("/content/drive/MyDrive/Snotel_Example/" + dataset):
    os.mkdir("/content/drive/MyDrive/Snotel_Example/" + dataset)
  if not os.path.exists("/content/drive/MyDrive/Snotel_Example/" + dataset + "/" + variable):
    os.mkdir("/content/drive/MyDrive/Snotel_Example/" + dataset + "/" + variable)

  print(df)
  # # Write Pandas data frame to CSV in Google Drive
  df.to_csv("/content/drive/MyDrive/Snotel_Example/" + dataset + "/" + variable + "/" + siteName + ".csv")
  plt.savefig("/content/drive/MyDrive/Snotel_Example/" + dataset + "/" + variable + "/" + siteName + ".png",  bbox_inches='tight')

In [ ]:
#Define Variables
dataset = "SNODAS"
variable = "SWE"
start_date = "2003-10-01"
end_date = "2023-02-25"
variableC = "SWE (mm)"

# Loop over allotment names to timeseries CSVs to Google Drive
for i in siteName:
  write_timeseries(i, dataset, variable, start_date, end_date, variableC)

Testing API call before automated export above

In [ ]:
#Make SWE timeseries request for all locations

# Endpoint
endpoint9 = 'timeseries/native/points'

# Set up parameters for API call
params9 = {
    'dataset':'SNODAS',
    'variable':'Snow_Depth',
    'start_date':'2003-10-01' ,
    'end_date':'2023-02-25',
    'coordinates': '[[-111.41, 34.46]]',
    'area_reducer':'mean'
}
# Send request to the API
r9 = requests.get(root_url + endpoint9, params=params9, headers=headers, verify=False)
response9 = r9.json()
print(response9)


In [ ]:
#response (may need to unpack with [] around timeseries the first time)
[timeseries] = response9

#Select data
data = timeseries['Data']

# Convert to dataframe
df = pd.DataFrame.from_dict(data)
print(df)

In [ ]:
#Process timeseries data 
#Filter out non-available data i.e. values of -9999.000
df = df[df['Snow_Depth (mm)']>-100]

#Format dates for plotting
df['Date'] = pd.to_datetime(df['Date'])

#Set x values to date variable
date = df['Date']

#Set y values to value variable
value = df['Snow_Depth (mm)']

In [ ]:
#Plot timeseries data
#Generate graph of long term blend values across date range
fig, ax = plt.subplots(figsize=(18,6))
ax.plot(date, value, label = siteName[0],)  

#Add x-axis label
plt.xlabel("Dates")
                      
#Add y-axis label
plt.ylabel("Snow_Depth (mm)")
           
#Add Title
plt.title(dataset + " " + variable  + " " + start_date +" through " + end_date)

#Add Legend 
plt.legend()

#Export Graph
plt.savefig('Depth_Plot.png')
           
#Show Graph
plt.show()